# Lab 7: 코드 확인

## 코드 확인 준비

먼저 이 실습 전반에 걸쳐 사용할 **컨텍스트 정보**를 가져오갰습니다. 

- **Start** 버튼을 클릭하여 이 노트북을 활성화합니다.

- 아래 Python 셀을 실행합니다.

#### :warning: 이 노트북에서 새 세션이 시작될 때마다, 이후 셀에서 사용할 "변수"를 설정하려면 아레 셀을 다시 실행해야 합니다. :warning:

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()
user = session.get_current_user().strip('"')
your_db = user + '_DB'
print('Your current CONTEXT information:')
print('---------------------------------')
print(session)
print('Your current USER is ' + user)

### INFORMATION_SCHEMA를 사용한 메타데이터 쿼리 🥋

"메타데이터"는 "데이터에 대한 데이터"를 의미합니다. 

모든 Snowflake 데이터베이스에 생성되는 `INFORMATION_SCHEMA`는 메타데이터를 보관합니다. 즉, 데이터베이스, 스키마, 테이블, 뷰 등의 수에 대한 통계와 오브젝트 이름 및 기타 세부정보를 포함하고 있습니다.

`INFORMATION_SCHEMA`를 사용하면 작업을 이중으로 확인하고 작업이 올바르게 완료되었는지 검증할 수 있습니다.

지금까지 작업했던 **(animal)_GARDEN_PLANTS** 데이터베이스의 모든 스키마를 조회하기 위해, 해당 데이터베이스의 `INFORMATION_SCHEMA`를 쿼리해 보겠습니다.

In [ ]:
SELECT * 
FROM {{user}}_garden_plants.information_schema.schemata;

### 작업 확인을 위한 코드 실행 🥋 

**(animal)_GARDEN_PLANTS** 데이터베이스에 3개의 스키마를 설정하고, 하나의 스키마를 삭제하라는 요청을 받았었습니다. 해당 작업이 완료되었는지 확인하기 위해 코드를 실행해 봅시다.

💡 **팁**: 다음 확인 코드는 [CTE (common table expression)](https://docs.snowflake.com/ko/user-guide/queries-cte) 구조를 사용합니다. CTE는 SQL 구문 내에서 사용할 수 있는 "temporary view"로 생각할 수 있습니다. CTE는 복잡한 SQL 문을 나누어 가독성을 높이고 관리하기 쉽게 만드는 데 특히 유용합니다. 논리를 별도의 재사용 가능한 파트로 구성함으로써 쿼리 구조를 단순화하고, 명확성을 높이며, 유지 관리를 용이하게 합니다.

In [ ]:
WITH schema_check_1 AS (
    -- do the following three schemas exist?
    SELECT COUNT(*) AS count_1
    FROM {{user}}_garden_plants.information_schema.schemata
    WHERE schema_name IN ('FLOWERS','VEGGIES','FRUITS')
),
schema_check_2 AS (   
    -- the following schema SHOULD NOT exist (count of zero)
    SELECT COUNT(*) AS count_2
    FROM {{user}}_garden_plants.information_schema.schemata
    WHERE schema_name = ('PUBLIC')
) 
SELECT IFF((count_1=3) AND (count_2=0),'\u2705 Correct','\u26D4 Incorrect. Please review and try again') AS schema_check
from schema_check_1, schema_check_2;

## (animal)_GARDEN PLANTS 데이터베이스에 몇 개의 스키마가 있나요? 

### 내가 무엇을 잘못했을까요? 📓 

위의 쿼리를 실행했는데 실패가 보고되었나요? 다음과 같은 실수일 가능성이 있습니다:

- "VEGGIES" 대신 "WEGGIES"를 사용한 것처럼 스키마 이름에 오타가 있을 수 있습니다.

- (user)_GARDEN_PLANTS 대신 UTIL_DB에 처럼 잘못된 데이터베이스에 스키마를 생성했을 수도 있습니다.

- `(animal)_LEARNER_RL` 역할로 오브젝트를 생성했지만, 워크시트의 역할이 `PUBLIC`로 설정되어 있는 것처럼, 역할이 제대로 설정되지 않아 오브젝트를 볼 수 없을 수 있습니다. 

---

### 어떻게 수정할 수 있을까요? 📓 

**오타**: `ALTER SCHEMA (animal)_GARDEN_PLANTS.WEGGIES RENAME TO (animal)_GARDEN_PLANTS.VEGGIES;`

**잘못된 위치**: `ALTER SCHEMA DEMO_DB.VEGGIES RENAME TO (animal)_GARDEN_PLANTS.VEGGIES;`

**찾을 수 없음**: 워크시트의 역할 설정을 변경하거나 오브젝트의 소유권을 이전하세요. 

### 스키마 이름 확인 🥋 

이제 생성한 스키마의 이름이 올바른지 확인해 보겠습니다. 아래 코드를 실행했을 때 결과가 **3**개 행으로 반환되지 않는 다면, 스키마 이름이 다르게 지정되었을 가능성이 있습니다.

In [ ]:
SELECT schema_name 
FROM {{user}}_GARDEN_PLANTS.INFORMATION_SCHEMA.SCHEMATA
WHERE schema_name IN ('FLOWERS','FRUITS','VEGGIES');

## 작업 확인🔎

### :mag_right: Check 1 (OB01)

- Garden Plant 데이터베이스에 3개의 스키마(**FLOWERS**, **VEGGIES**, **FRUITS**)를 생성했나요?
- 작업을 확인하려면 채점용 Stored Procedure를 호출하세요.

In [ ]:
CALL common_db.resources.local_grader('OB01', '{{user}}');

### :mag_right: Check 2 (OB02) 🔎

- Garden Plant 데이터베이스의 **PUBLIC** 스키마를 삭제(Drop)했나요?
- 작업을 확인하려면 채점용 Stored Procedure를 호출하세요.

In [ ]:
CALL common_db.resources.local_grader('OB02', '{{user}}');

### :mag_right: Check 3 (OB03) 🔎

- Garden Plant 데이터베이스 **VEGGIES** 스키마에 **ROOT_DEPTH** 테이블을 생성했나요?
- 작업을 확인하려면 채점용 Stored Procedure를 호출하세요.

In [ ]:
CALL common_db.resources.local_grader('OB03', '{{user}}');

### Query History를 사용하여 테스트 결과 검토 📓

Snowflake는 시스템에서 실행된 쿼리와 구문의 기록을 보관하며, 이를 Query History라고 합니다. Query History는 사용자 인터페이스(UI)와 프로그래밍 방식을 통해 접근할 수 있습니다. Query History는 자신(또는 권한이 있는 다른 사용자)이 실행한 쿼리를 시간별로 확인할 수 있는 편리한 장소를 제공합니다.

Snowflake Notebook의 셀에서 Query History에 접근할 수 있습니다.

### Snowflake Notebook SQL 셀에서 Query History 접근하기 🥋

1. 방금 실행한 **Check 3** SQL 셀에서 쿼리 실행 시간 표시에 마우스 커서를 올려놓으세요.
1. **View run details** 메시지가 나타납니다.
1. 쿼리 실행 시간 표시를 클릭하면 대화 상자가 나타납니다.
1. 파란색 **ID** 필드 UUID를 클릭합니다. 이 필드는 Snowsight Query History 페이지로 연결됩니다.
1. 새 브라우저 창에서 Snowsight Query History 페이지가 열립니다.

![Access query history (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_query_history_1.png)

1. **Query Profile** 화면이 열리며, 여기에서 쿼리의 실행 단계를 검토할 수 있습니다.
1. 화면 상단의 **Query Details** 탭을 클릭합니다.
1. 쿼리 실행과 관련된 다양한 세부 정보를 검토합니다.
1. 페이지 하단의 **Results** 섹션을 검토합니다. 

![Query details (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_query_details_1.png)

:warning: Query Details(Query History)의 Result 섹션에서 Check 3에 대한 초록색 체크 표시 ✅ 를 확인하기 전까지는 다음 단계로 **진행하지 마세요**. :warning:

### Query History를 프로그래밍 방식으로 액세스하기 🥋

우리는 코드 사용을 통해 Query History 정보를 검색할 수 있으며, Python과 SQL 모두에서 사용할 수 있는 옵션이 있습니다.

`INFORMATION_SCHEMA`는 다양한 차원에서 Snowflake Query History를 쿼리할 수 있는 [테이블 함수](https://docs.snowflake.com/ko/sql-reference/functions/query_history) 모음을 포함하고 있습니다. 다음 예제에서는 `QUERY_HISTORY_BY_USER()`를 사용하여 지난 7일 동안 사용자(여러분!)가 제출한 쿼리를 반환하게 합니다.

우리는 **Lab 6: The Load Wizard 및 Snowflake Marketplace**에서 실행한 특정 `DELETE` 작업을 식별할 것입니다. 이 작업에서 **VEGETABLE_DETAILS** 테이블에서 단일 행을 제거했었습니다. (...plant_name = 'Spinach' AND root_depth_code = 'D').

In [ ]:
SELECT *
FROM TABLE(information_schema.query_history_by_user(
    USER_NAME => '{{user}}',
    RESULT_LIMIT => 10000
))
WHERE query_type = 'DELETE'
AND execution_status = 'SUCCESS'
ORDER BY end_time DESC
LIMIT 1;

이 작업의 Query ID를 SQL 변수에 저장하여 이번 실습에서 나중에 사용할 수 있도록 하겠습니다.

**:warning: `INFORMATION_SCHEMA.QUERY_HISTORY_BY_USER`와 그 변형은 데이터를 7일 동안만 보관합니다. 이 기간 외에 실행된 쿼리의 경우 결과를 반환하지 않습니다. 이 경우 Lab 6의 단계를 다시 완료해야합니다. :warning:**

In [ ]:
SET delete_query_id = (
    SELECT query_id
    FROM TABLE(information_schema.query_history_by_user(
        USER_NAME => '{{user}}',
        RESULT_LIMIT => 10000
    ))
    WHERE query_type = 'DELETE'
    AND execution_status = 'SUCCESS'
    ORDER BY end_time DESC
    LIMIT 1
);

SELECT $delete_query_id;

### :mag_right: Check 4 (OB04) 🔎

- 데이터베이스 **(animal)_UTIL_DB**에 정확히 2개의 스키마가 포함되어 있습니까?
- 작업을 확인하려면 채점용 Stored Procedure를 호출하세요.

In [ ]:
CALL common_db.resources.local_grader('OB04', '{{user}}');

### :mag_right: Check 5 (OB05) 🔎

- Garden Plant 데이티베이스의 **VEGGIES** 스키마에 **VEGETABLE_DETAILS** 테이블을 생성했습니까?
- 작업을 확인하려면 채점용 Stored Procedure를 호출하세요.

In [ ]:
CALL common_db.resources.local_grader('OB05', '{{user}}');

### Check 6 (OB06) 🔎

- **ROOT_DEPTH** 테이블에 **3**개의 행이 있나요?
- 작업을 확인하려면 채점용 Stored Procedure를 호출하세요.

In [ ]:
CALL common_db.resources.local_grader('OB06', '{{user}}');  

### Check 7 (OB07) 🔎

- **VEGETABLE_DETAILS** 테이블에 **41** 개의 행이 있나요?
- 작업을 확인하려면 채점용 Stored Procedure를 호출하세요.

In [ ]:
CALL common_db.resources.local_grader('OB07', '{{user}}'); 

## Time Travel 📓

Snowflake Time Travel 기능은 특정 기간 내의 과거 데이터(즉, 변경되거나, 삭제된 데이터)에 액세스할 수 있도록 해줍니다. Time Travel을 지원하기 위해 여러 [SQL 확장](https://docs.snowflake.com/ko/user-guide/data-time-travel#time-travel-sql-extensions)기능이 제공됩니다.

데이터가 삭제되거나 데이터를 포함한 오브젝트가 삭제(drop)되는 것을 포함하여 테이블의 데이터가 변경된 경우, Snowflake는 업데이트 이전의 데이터 상태를 보존합니다. `DATA_RETENTION_TIME_IN_DAYS`라는 파라미터는 이 과거 데이터가 보존되는 일수를 지정하며, 이를 통해 Time Travel 작업(`SELECT`, `CREATE` … `CLONE`, `UNDROP`)을 수행할 수 있습니다.

### **VEGETABLE_DETAILS** 테이블의 과거 버전 조회 🥋

**Lab 6: The Load Wizard 및 Snowflake Marketplace** 초반에 **VEGETABLE_DETAILS** 테이블을 생성할 때, 데이터 보존 기간이 7일로 설정되어 있었습니다. 이는 테이블에서 수행된 모든 변경 사항이 7일 동안 유지됨을 의미합니다. 이를 통해 테이블에 관련하여 특정 시점이나 작업 이전 상태의 데이터를 확인할 수 있도록 해줍니다.

**Lab 6**에서, **VEGETABLE_DETAILS** 테이블에서 두 번째 Spinach 행을 삭제했습니다. 이번 실습의 앞부분에서 해당 작업과 연관된 Query ID를 식별하고, 이를 로컬 변수 `$delete_query_id`에 저장했습니다. Time Travel을 사용하여 이 작업 이전의 데이터를 확인할 수 있습니다.

아래 코드를 실행하여 테이블의 `DELETE` 작업 이전의 Spinach 행과 현재 버전에 있는 Spinach 데이터를 **UNION**해서 조회합니다. Time Travel을 사용하여 과거 데이터를 쿼리하기 위해 사용된 [특수 구문](https://docs.snowflake.com/ko/user-guide/data-time-travel#querying-historical-data)을 참고하세요: `BEFORE( STATEMENT => $delete_query_id )`

In [ ]:
-- Time Travel query 
SELECT plant_name, root_depth_code, 'HISTORICAL (Time Travel)' as table_version 
FROM {{user}}_garden_plants.veggies.vegetable_details
BEFORE( STATEMENT => $delete_query_id )
WHERE plant_name = 'Spinach'

UNION

-- current version query
SELECT plant_name, root_depth_code, 'CURRENT' as table_version
FROM {{user}}_garden_plants.veggies.vegetable_details
WHERE plant_name = 'Spinach';

## 지식 테스트 :mag_right:

아래 Python 셀을 실행하여 Streamlit 기반의 위젯을 표시하고 Snowflake 인터페이스에 대한 질문에 답하세요. 지금은 이 코드가 무엇을 하는지 이해할 필요가 없습니다. 그냥 코드를 실행하세요.

질문에 올바르게 답해야 다음 섹션으로 진행할 수 있습니다.

In [ ]:
import streamlit as st
st.divider()
question = "메타데이터의 정의는 무엇입니까?"
options = ["아래 보기 중 고르세요...",
           "A) 메타에 관한 데이터입니다.", 
           "B) 다른 데이터 위에 있는 데이터입니다.", 
           "C) 데이터에 관한 데이터입니다."]

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...":
        ''
    else:
        answer = '44f0134bdf8040c4f1274e360a615f40'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

In [ ]:
st.divider()
question = "Snowflake는 일부 메타데이터를 어디에 저장합니까?"
options = ["아래 보기 중 고르세요...",
           "A) 각 계정의 INFORMATION_DB 데이터베이스에 저장합니다.", 
           "B) 각 데이터베이스의 METADATA_SCHEMA에 저장합니다.", 
           "C) 각 데이터베이스의 INFO_METADATA 스키마에 저장합니다.",
           "D) 각 데이터베이스의 INFORMATION_SCHEMA 스키마에 저장합니다."]

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...":
        ''
    else:
        answer = '447a7f80b1d7200de9d1ffafdf15e61d'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

In [ ]:
st.divider()
question = "생성한 스키마를 확인하려면 직접 눈으로 확인하면 안되는 이유는 무엇입니까?"
options = ["아래 보기 중 고르세요...",
           "A) 철자가 틀렸을 경우 이름 문제를 눈치채지 못할 수 있지만, 코드를 통해 확인하면 이를 발견할 수 있기 때문", 
           "B) 어떤 이유로 자동화를 통해 확인하고 싶을 수 있기 때문", 
           "C) 수동 확인이 코딩보다 훨씬 더 재미있기 때문"]

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...":
        ''
    else:
        answer = '5c6dfae473420b365f5f247ee6c65487'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

## 다음 단계

채점 확인(1-7)이 통과되었고, **Knowledge Test** 질문에 올바르게 답했다면, Snowflake 강사의 안내에 따라 다음 노트북으로 진행하세요.